In [1]:
import time
import numpy as np
from xarm.wrapper import XArmAPI

SDK_VERSION: 1.13.30


In [2]:

arm = XArmAPI('192.168.1.217')
arm.connect()


ROBOT_IP: 192.168.1.217, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3


In [6]:

def next_cmd(t, path_function, time_step=0.01):
    new_z = path_function(t+time_step)
    return new_z


def fish_motion(arm:XArmAPI, amplitude, frequency, yaw_amplitude, yaw_frequency, phase_shift, duration):
    arm.set_mode(1)
    arm.set_state(0)
    time.sleep(0.1)
    _, (start_x, start_y, start_z, start_roll, start_pitch, start_yaw) = arm.get_position()
    _, (current_x, current_y, current_z, current_roll, current_pitch, current_yaw) = arm.get_position()
    path = lambda t: amplitude * np.sin(2 * np.pi * frequency * t)
    yaw = lambda t: yaw_amplitude * np.sin(2 * np.pi * yaw_frequency * t + phase_shift)
    steps = []
    time_step = 0
    t0 = t = time.perf_counter()
    while time.perf_counter()-t0 < duration:
        t = time.perf_counter()-t0
        i = next_cmd(t, path)
        j = next_cmd(t, yaw)
        arm.set_servo_cartesian([start_x, start_y+i, start_z, start_roll, start_pitch, start_yaw+j])
        steps.append([t, i])

    _, (current_x, current_y, current_z, current_roll, current_pitch, current_yaw) = arm.get_position()
    while abs(current_y - start_y) > 0.1:
        _, (current_x, current_y, current_z, current_roll, current_pitch, current_yaw) = arm.get_position()
        t = time.perf_counter()-t0
        i = next_cmd(t, path)
        j = next_cmd(t, yaw)
        arm.set_servo_cartesian([start_x, start_y+i, start_z, start_roll, start_pitch, start_yaw+j])
        steps.append([t, i])
    _, (current_x, current_y, current_z, current_roll, current_pitch, current_yaw) = arm.get_position()
    
    arm.set_servo_cartesian([start_x, start_y, start_z, start_roll, start_pitch, start_yaw])
    
    return steps



In [17]:

amplitude=5
frequency=1
duration = 5

yaw_amplitude=5
yaw_frequency=frequency
phase_shift = 0

steps = fish_motion(arm, amplitude, frequency, yaw_amplitude, yaw_frequency, phase_shift, duration)


In [12]:
arm.set_state(0)
arm.get_state()


[set_state], xArm is ready to move


(0, 2)

In [ ]:

arm.disconnect()